This is a **private notebook** to solve the optimization part in the **Shell.ai Hackathon 2023**. **Important, do not publish this notebook or its results anywhere, for private use only**.

# 1. Data loading

In [2]:
!pip3 install mip

In [3]:
import pandas as pd
import numpy as np
import os
from mip import Model, xsum, minimize, OptimizationStatus, BINARY
import matplotlib.pyplot as plt

SYNTH_DATA_PATH = '../data'     
# OUT_SYNTH_DATA_PATH = '/kaggle/working'
FORECAST_FILE = 'Biomass_History_Synthetic.csv'
DISTANCE_FILE = 'Distance_Matrix_Synthetic.csv'

d_matrix = pd.read_csv(os.path.join(SYNTH_DATA_PATH, DISTANCE_FILE), 
                       index_col=0)
d_matrix = d_matrix.values

In [4]:
MAX_DISTANCE = 250          # Maximum considered distance between harvesting site and depot
posible_locations = np.sum(d_matrix <= MAX_DISTANCE)
print('% distances over the selected max: ', np.sum(d_matrix > MAX_DISTANCE) / (2418 * 2418) * 100)

% distances over the selected max:  65.03448768917431


In [5]:
d_matrix.shape[0] ** 2

5846724

In [6]:
arr = np.array([[1,2,3],[4,5,6],[7,5,9]])
arr

array([[1, 2, 3],
       [4, 5, 6],
       [7, 5, 9]])

In [7]:
np.where(d_matrix <= MAX_DISTANCE)[1].shape[0] / d_matrix.shape[0] ** 2

0.34965512310825686

In [8]:
d_matrix.max()

916.1131

In [10]:
2418 ** 2

5846724

In [9]:
MAX_DISTANCE = 20.
idxs = np.where(d_matrix <= MAX_DISTANCE)
df_arcs_ij = pd.DataFrame(idxs[1].reshape(-1,1), index=idxs[0]).reset_index()
df_arcs_ij.columns = ['i', 'j']

df_arcs_jk = df_arcs_ij.copy()
df_arcs_jk.columns = ['j', 'k']

df_arcs = pd.merge(df_arcs_ij['i'], df_arcs_jk, left_on='i', right_on='j').values
df_arcs.shape

(116932, 3)

In [19]:
df_arcs_ij.values

array([[   0,    0],
       [   0,    1],
       [   0,   15],
       ...,
       [2416, 2417],
       [2417, 2416],
       [2417, 2417]], dtype=int64)

In [24]:
df_arcs.values

array([[   0,    0,    0],
       [   0,    0,    1],
       [   0,    0,   15],
       ...,
       [2417, 2417, 2417],
       [2417, 2417, 2416],
       [2417, 2417, 2417]], dtype=int64)

In [22]:
df_arcs_ij.duplicated().sum()

0

In [ ]:
from itertools import product
ls_arcs = list(product(ls_nodes, ls_nodes))

In [4]:
N = 30#1000 #2418   # Use 30 for a small problem (optimal solution: to be computed)
TRANSPORT_FACTOR_A = 0.001
d_matrix_cost = TRANSPORT_FACTOR_A * d_matrix[:N, :N]

df_fc = pd.read_csv(os.path.join(SYNTH_DATA_PATH, FORECAST_FILE))
df_fc = df_fc.iloc[:N, :]

ls_j = range(len(d_matrix_cost))  # Posible locations for depots and refineries
year = 2018                       # Not sure if this is necessary
cap_b_j = 20000                   # Maximum depot capacity
cap_p_j = 100000                  # Maximum production capacity
n_refineries = 5                  # Number of refineries
n_depots = 25                     # Number of depots

# Get the forecasted biomass for year 2018 of all the positions
d_bio_18 = df_fc.loc[:, '2018']
total_fc_18 = d_bio_18.sum()
d_bio_18 = d_bio_18.to_dict()
print("Forecasted biomass for year 2018: ", total_fc_18)


d_bio_19 = df_fc.loc[:, '2019']
total_fc_19 = d_bio_19.sum()
d_bio_19 = d_bio_19.to_dict()
print("Forecasted biomass for year 2019: ", total_fc_19)

# Get the solution for the optimization problem
m = Model(sense=minimize)
m.threads = -1

# Variables: biomass b_{i, 0}
# 1. All values (forecasted biomass, biomass demand-supply, pellet demand-supply) must be
# greater than or equal to zero.

# Compute the distances that are less than the desired limit
accepted_distances = []           # Accepted [i, j] pairs according to distance (i is source, j is destination)
accepted_destin_dict = {}         # Dict of possible destinations for harvested sources (i is key, j are the values)
accepted_source_dict = {}         # Dict of possible sources for depots destionations (j is key, i are the values)
for i in range(len(d_matrix_cost)):
    accepted_destin_dict[i] = []
    accepted_source_dict[i] = []
for i in range(len(d_matrix_cost)):
    for j in range(len(d_matrix_cost)):
        if d_matrix[i, j] <= MAX_DISTANCE:
            accepted_distances.append((i, j))
            accepted_destin_dict[i].append(j)
            accepted_source_dict[j].append(i)

b_18 = [m.add_var(name=f'b_2018_{i}_{j}', lb=0) for i, j in accepted_distances]
print(f"Variables b_2018 go from {b_18[0].name} to {b_18[-1].name}")

b_19 = [m.add_var(name=f'b_2019_{i}_{j}', lb=0) for i, j in accepted_distances]
print(f"Variables b_2019 go from {b_19[0].name} to {b_19[-1].name}")

p_18 = [m.add_var(name=f'p_2018_{i}_{j}', lb=0) for i in range(len(d_matrix_cost)) for j in ls_j]
print(f"Variables p_2018 go from {p_18[0].name} to {p_18[-1].name}")

p_19 = [m.add_var(name=f'p_2019_{i}_{j}', lb=0) for i in range(len(d_matrix_cost)) for j in ls_j]
print(f"Variables p_2019 go from {p_19[0].name} to {p_19[-1].name}")

x = [m.add_var(name=f'x_{j}', var_type=BINARY) for j in ls_j]
print(f"Variables x go from {x[0].name} to {x[-1].name}")

r = [m.add_var(name=f'r_{j}', var_type=BINARY) for j in ls_j]
print(f"Variables r go from {r[0].name} to {r[-1].name}")

# Constraints:
# 2. The amount of biomass procured for processing from each harvesting site ′𝑖𝑖′ must be less than
# or equal to that site’s forecasted biomass.
print('Adding constraints number 2')
for i in range(len(d_matrix_cost)):
    m += xsum(m.var_by_name(f'b_2018_{i}_{j}') for j in accepted_destin_dict[i]) <= d_bio_18[i]
    m += xsum(m.var_by_name(f'b_2019_{i}_{j}') for j in accepted_destin_dict[i]) <= d_bio_19[i]

print('Adding constraints number 3-4')
for j in ls_j:
    # 3-4. Can't transport more than storage limit
    m += xsum(m.var_by_name(f'b_2018_{i}_{j}') for i in accepted_source_dict[j]) <= cap_b_j * x[j]
    m += xsum(m.var_by_name(f'b_2019_{i}_{j}') for i in accepted_source_dict[j]) <= cap_b_j * x[j]
    m += xsum(m.var_by_name(f'p_2018_{i}_{j}') for i in ls_j) <= cap_p_j * r[j]
    m += xsum(m.var_by_name(f'p_2019_{i}_{j}') for i in ls_j) <= cap_p_j * r[j]

# 8. Total amount of biomass entering each preprocessing depot is equal to the total amount of
# pellets exiting that depot (within tolerance limit of 1e-03)
print('Adding constraints number 8')
# We don't really know if two xsum is worst than a single xsum over all data
for j in ls_j:
    m += xsum(m.var_by_name(f'b_2018_{i}_{j}') for i in accepted_source_dict[j]) + xsum(-m.var_by_name(f'p_2018_{j}_{k}') for k in ls_j) <=\
          .001 * x[j]
    m += xsum(-m.var_by_name(f'b_2018_{i}_{j}') for i in accepted_source_dict[j]) + xsum(m.var_by_name(f'p_2018_{j}_{k}') for k in ls_j) <=\
          .001 * x[j]
    
    m += xsum(m.var_by_name(f'b_2019_{i}_{j}') for i in accepted_source_dict[j]) + xsum(-m.var_by_name(f'p_2019_{j}_{k}') for k in ls_j) <=\
          .001 * x[j]
    m += xsum(-m.var_by_name(f'b_2019_{i}_{j}') for i in accepted_source_dict[j]) + xsum(m.var_by_name(f'p_2019_{j}_{k}') for k in ls_j) <=\
          .001 * x[j]

print('Adding constraints number 5')
# 5. Number of depots should be less than or equal to 25.
m += xsum(x[j] for j in ls_j) <= n_depots

print('Adding constraints number 6')
# 6. Number of refineries should be less than or equal to 5.
m += xsum(r[j] for j in ls_j) <= n_refineries

print('Adding constraints number 7')
# 7. At least 80% of the total forecasted biomass must be processed by refineries each year
m += xsum(m.var_by_name(f'p_2018_{i}_{j}') for i in ls_j for j in ls_j)\
    >= 0.8 * total_fc_18
m += xsum(m.var_by_name(f'p_2019_{i}_{j}') for i in ls_j for j in ls_j)\
    >= 0.8 * total_fc_19

m.objective = minimize(xsum(d_matrix_cost[i, j] * (m.var_by_name(f'b_2018_{i}_{j}') + m.var_by_name(f'b_2019_{i}_{j}')) \
                            for i in range(len(d_matrix_cost)) for j in accepted_destin_dict[i]) + \
                       xsum(d_matrix_cost[i, j] * (m.var_by_name(f'p_2018_{i}_{j}') + m.var_by_name(f'p_2019_{i}_{j}')) \
                            for i in ls_j for j in ls_j) + \
                       xsum(2*cap_b_j*x[j] + 2*cap_p_j*r[j] for j in ls_j) + \
                       xsum(-m.var_by_name(f'b_2018_{i}_{j}') - m.var_by_name(f'b_2019_{i}_{j}') \
                            for j in ls_j for i in accepted_source_dict[j]) + \
                       xsum(-m.var_by_name(f'p_2018_{i}_{j}') - m.var_by_name(f'p_2019_{i}_{j}') \
                            for i in ls_j for j in ls_j) \
                      )

Forecasted biomass for year 2018:  1488.016473495
Forecasted biomass for year 2019:  1766.976034022
Variables b_2018 go from b_2018_0_0 to b_2018_29_29
Variables b_2019 go from b_2019_0_0 to b_2019_29_29
Variables p_2018 go from p_2018_0_0 to p_2018_29_29
Variables p_2019 go from p_2019_0_0 to p_2019_29_29
Variables x go from x_0 to x_29
Variables r go from r_0 to r_29
Adding constraints number 2
Adding constraints number 3-4
Adding constraints number 8
Adding constraints number 5
Adding constraints number 6
Adding constraints number 7


In [5]:
print('Number of constraints: ', m.num_rows)                  # number of rows (constraints) in the model
print('Number of variables: ', m.num_cols)                    # number of columns (variables) in the model
print('Number of integer variables: ', m.num_int)             # number of integer variables in the model
print('Number of non-zeros in constraint matrix: ', m.num_nz) # number of non-zeros in the constraint matrix

Number of constraints:  304
Number of variables:  3660
Number of integer variables:  60
Number of non-zeros in constraint matrix:  14700


Some information about the problem reduction:
Results for the full 2418 locations problem (no model size reduction):
- Number of constraints:  24,184
- Number of variables:  23,391,732
- Number of integer variables:  4,836
- Number of non-zeros in constraint matrix:  93,571,764

Results for the full 2418 locations problem (with model size reduction MAX_DISTANCE=250):
- Number of constraints:  24184
- Number of variables:  15,786,958
- Number of integer variables:  4,836
- Number of non-zeros in constraint matrix:  63,152,668

Results for 50 locations problem (no model size reduction):
- Number of constraints:  504
- Number of variables:  10,100
- Number of integer variables:  100
- Number of non-zeros in constraint matrix:  40,500

Results for 50 locations problem (with model size reduction MAX_DISTANCE=250). Note that the problem size is the same as all the locations are within range:
- Number of constraints:  504
- Number of variables:  10,100
- Number of integer variables:  100
- Number of non-zeros in constraint matrix:  40,500

Results for 750 locations problem (no model size reduction), this problem can be solved in reasonable time (18 minutes):
- Number of constraints:  7,504
- Number of variables:  2,251,500
- Number of integer variables:  1,500
- Number of non-zeros in constraint matrix:  9,007,500

Results for 750 locations problem (with model size reduction MAX_DISTANCE=250):
- Number of constraints:  7,504
- Number of variables:  1,840,222
- Number of integer variables:  1,500
- Number of non-zeros in constraint matrix:  7,362,388

Results for 1000 locations problem (no model size reduction):
- Number of constraints:  10,004
- Number of variables:  4,002,000
- Number of integer variables:  2,000
- Number of non-zeros in constraint matrix:  16,010,000

Results for 1000 locations problem (with model size reduction MAX_DISTANCE=250):
- Number of constraints:  10,004
- Number of variables:  3,155,932
- Number of integer variables:  2,000
- Number of non-zeros in constraint matrix:  12,625,728


In [6]:
print("Solve")
# Solve the problem
# m.max_gap = 0.1
# m.threads = -1

status = m.optimize() # max_seconds=100

print(status)
# Check the status and show the solutions
if status == OptimizationStatus.OPTIMAL:
    print('optimal solution cost {} found'.format(m.objective_value))
elif status == OptimizationStatus.FEASIBLE:
    print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
elif status in [OptimizationStatus.NO_SOLUTION_FOUND, OptimizationStatus.INFEASIBLE]:
    print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))

if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    #print('solution:')
    d_sol = {}
    for v in m.vars:
        d_sol.update({v.name: v.x})
    #print("Solution: ", d_sol)
    df_sol = pd.DataFrame.from_dict(d_sol, orient='index', columns=['biomass'])
    df_sol.to_csv(os.path.join(OUT_SYNTH_DATA_PATH, 'solution.csv'))

Solve
Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming relaxation problem using Dual Simplex

Coin0506I Presolve 304 (0) rows, 3660 (0) columns and 14700 (0) elements
Clp0014I Perturbing problem by 0.001% of 20916.457 - largest nonzero change 0.018922632 ( 0.17406027%) - largest zero change 0
Clp0000I Optimal - objective value -0.00019404254
Clp0032I Optimal objective -0.0001940425357 - 379 iterations time 0.012
Clp1000I Unscaled problem has primal infeasibilities

Starting MIP optimization
Cgl0004I processed model has 304 rows, 3660 columns (60 integer (60 of which binary)) and 14700 elements
Coin3009W Conflict graph built in 0.001 seconds, density: 0.000%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0045I Nauty did not find any useful orbits in time 0.109842
Cbc0038I Initial state - 48 integers unsatisfied sum - 0.826666
Cbc0038I Pass   1: suminf.    0.22667 (2) obj. 32238.2 iterations 205
Cb

In [7]:
# Sanity check for all quantities
sent_biomass_to_depots_18 = df_sol.loc[df_sol.index.str.startswith('b_2018'), 'biomass'].sum()
sent_pellets_to_refinery_18 = df_sol.loc[df_sol.index.str.startswith('p_2018'), 'biomass'].sum()
print('Forecasted biomass for year 2018: ', total_fc_18)
print('Biomass transported to depots for year 2018: ', sent_biomass_to_depots_18)
print('Biomass transported to depots for year 2018: ', sent_pellets_to_refinery_18)
print()
sent_biomass_to_depots_19 = df_sol.loc[df_sol.index.str.startswith('b_2019'), 'biomass'].sum()
sent_pellets_to_refinery_19 = df_sol.loc[df_sol.index.str.startswith('p_2019'), 'biomass'].sum()
print("Forecasted biomass for year 2019: ", total_fc_19)
print('Biomass transported to depots for year 2019: ', sent_biomass_to_depots_19)
print('Biomass transported to depots for year 2019: ', sent_pellets_to_refinery_19)

Forecasted biomass for year 2018:  1488.016473495
Biomass transported to depots for year 2018:  1488.016473495
Biomass transported to depots for year 2018:  1488.0174734949999

Forecasted biomass for year 2019:  1766.976034022
Biomass transported to depots for year 2019:  1766.9760340219998
Biomass transported to depots for year 2019:  1766.9770340219998


In [8]:
# Compute total cost
total_cost = float(xsum(d_matrix_cost[i, j] * (df_sol.loc[f'b_2018_{i}_{j}'][0] + df_sol.loc[f'b_2019_{i}_{j}'][0]) \
                            for i in range(len(d_matrix_cost)) for j in accepted_destin_dict[i])) + \
                       float(xsum(d_matrix_cost[i, j] * (df_sol.loc[f'p_2018_{i}_{j}'][0] + df_sol.loc[f'p_2019_{i}_{j}'][0]) \
                            for i in ls_j for j in ls_j)) + \
                       float(xsum(2*cap_b_j*x[j] + 2*cap_p_j*r[j] for j in ls_j)) + \
                       float(xsum(- df_sol.loc[f'b_2018_{i}_{j}'][0] - df_sol.loc[f'b_2019_{i}_{j}'][0] \
                            for j in ls_j for i in accepted_source_dict[j])) + \
                       float(xsum(- df_sol.loc[f'p_2018_{i}_{j}'][0] - df_sol.loc[f'p_2019_{i}_{j}'][0] \
                            for i in ls_j for j in ls_j))
print('Final transport and underuse cost: ', total_cost)

Final transport and underuse cost:  233607.49178295932


In [9]:
# This code works
# underuse_cost = float(xsum(2*cap_b_j*x[j] + 2*cap_p_j*r[j] for j in ls_j)) + \
#                        float(xsum(- df_sol.loc[f'b_2018_{i}_{j}'][0] - df_sol.loc[f'b_2019_{i}_{j}'][0] \
#                             for j in ls_j for i in accepted_source_dict[j])) + \
#                        float(xsum(- df_sol.loc[f'p_2018_{i}_{j}'][0] - df_sol.loc[f'p_2019_{i}_{j}'][0] \
#                             for i in ls_j for j in ls_j))
# print(underuse_cost)